# Import necessary packages if not already installed

In [ ]:
!pip install mlx-lm-lora mlx-lm datasets

# Import your needed modules

In [ ]:
from mlx_lm_lora.trainer.sft_trainer import SFTTrainingArgs, train_sft
from mlx_lm_lora.trainer.datasets import CacheDataset, TextDataset
from mlx_lm_lora.utils import fuse_model

from datasets import load_dataset, concatenate_datasets
from huggingface_hub import create_repo, HfApi

from mlx_lm.tuner.utils import linear_to_lora_layers, print_trainable_parameters
from mlx_lm.tuner.callbacks import TrainingCallback
from mlx_lm.utils import load, save_config

import mlx.optimizers as optim

from pathlib import Path
import math

# Define the Args

In [31]:
hf_token = "" # <-- Add you HF Token here

model_name = "mlx-community/Josiefied-Qwen3-0.6B-abliterated-v1-4bit"
new_model_name = "new-model"
user_name = "mlx-community"

adapter_path = "path/to/adapters"
max_seq_length = 512
num_layers = 12
lora_parameters = {"rank": 8, "dropout": 0.0, "scale": 10.0}

dataset_names = [
    "digitalpipelines/wizard_vicuna_70k_uncensored"
]
dataset_samples = 1000

# Load the model

In [ ]:
model, tokenizer = load(model_name)

# Convert to LoRA

In [ ]:
model.freeze()

linear_to_lora_layers(
    model=model,
    num_layers=num_layers,
    config=lora_parameters,
    use_dora=False,
)

print_trainable_parameters(model)

# Define the Optimizer

In [4]:
opt = optim.AdamW(learning_rate=1e-5)

# Load and Preprocess your Dataset

In [26]:
system_prompt = """You are **J.O.S.I.E.**, an advanced super-intelligent AI Assistant created by a 25 year old man named **Gökdeniz Gülmez**."""

def format_prompts_func(sample):
    this_conversation = sample["conversations"]

    if isinstance(this_conversation, list):
        conversation = []
        conversation.append({"role": "system", "content": system_prompt})
        for turn in this_conversation:
            if turn["from"] == "human":
                conversation.append({"role": "user", "content": turn['value']})
            elif turn["from"] == "gpt":
                conversation.append({"role": "assistant", "content": turn['value']})
        
    sample["text"] = tokenizer.apply_chat_template(
        conversation=conversation,
        add_generation_prompt=False,
        enable_thinking=False, # <- Only for Qwen models
        tokenize=False
    )
    return sample

datasets = [load_dataset(name)["train"] for name in dataset_names]
combined_dataset = concatenate_datasets(datasets)

if dataset_samples is not None:
    combined_dataset = combined_dataset.select(range(dataset_samples))

full_dataset = combined_dataset.map(format_prompts_func,)

train_dataset, valid_dataset = full_dataset.train_test_split(test_size=0.01, seed=42).values() # Split the full dataset into 99# Train and 1% Validation

In [ ]:
print(train_dataset[0]["text"])

# 📦 Make the Dataset for the trainer

You have multiple dataset wrappers available depending on your training format:

- **`TextDataset`**: Use this when your data is a plain string under a specific key like `"text"`.
- **`CompletionsDataset`**: Use when you have separate fields like `prompt` and `completion`.
- **`ChatDataset`**: Use this for structured chat data like Alpaca-style, ShareGPT, etc.

Be sure to import the wanted Dataset classes above.

In [ ]:
train_set = TextDataset(train_dataset, tokenizer, text_key='text')
valid_set = TextDataset(valid_dataset, tokenizer, text_key='text')

# Make the Adapter Folder and save the configs for loading later

In [ ]:
args = {
    "lora_parameters": lora_parameters,
    "num_layers": num_layers,
}

adapter_path = Path(adapter_path)
adapter_path.mkdir(parents=True, exist_ok=True)

adapter_file = adapter_path / "adapters.safetensors"
save_config(args, adapter_path / "adapter_config.json")

# Start training

In [ ]:
batch_size = 4
epochs = 2

num_samples = len(train_set)
batches_per_epoch = math.ceil(num_samples / batch_size)
iters = epochs * batches_per_epoch
print(f"[INFO] Calculated {iters} iterations from {epochs} epochs (dataset size: {num_samples}, batch size: {batch_size})")

train_sft(
    model=model,
    args=SFTTrainingArgs(
        batch_size=batch_size,
        iters=iters,
        val_batches=1,
        steps_per_report=20,
        steps_per_eval=50,
        steps_per_save=50,
        adapter_file=adapter_path,
        max_seq_length=max_seq_length,
        grad_checkpoint=True,
    ),
    optimizer=opt,
    train_dataset=CacheDataset(train_set),
    val_dataset=CacheDataset(valid_set),
    training_callback=TrainingCallback()
)

# Fuse the model with the trained adapters and save the new model

In [ ]:
fuse_model(
    model=model,
    tokenizer=tokenizer,
    save_path=new_model_name,
    adapter_path=adapter_path,
    de_quantize=False,
    export_gguf=False,
    gguf_path=f"{new_model_name}/model.gguf",
)

# Create the README

In [ ]:
readme_file = f"""---
tags:
- mlx
- lora
- text-generation
- fine-tuning
base_model: {model_name}
pipeline_tag: text-generation
---

# LoRA Fine-Tuned Model: `{user_name}/{new_model_name}`

This model is a LoRA fine-tuned version `{model_name}`, with the [`mlx-lm-lora`](https://github.com/Goekdeniz-Guelmez/mlx-lm-lora) training package on Apple Silicon using MLX.

---

## 🧾 Model Details

- **Model name:** {new_model_name}
- **Base model:** {model_name}
- **Fine-tuning method:** LoRA
- **Training package:** [`MLX-LM-LORA`](https://github.com/Goekdeniz-Guelmez/mlx-lm-lora)
- **Model type:** {model.args.model_type}
- **Author:** None

---

## 💡 Recommended System Prompt

```text
{system_prompt}
```
"""

new_readme_path = f"{new_model_name}/README.md"
with open(new_readme_path, "w") as new_readme_file:
    new_readme_file.write(readme_file)

# Upload it to HugginFace

In [ ]:
api = HfApi(token=hf_token)
create_repo(
  repo_id = f"{user_name}/{new_model_name}",
  repo_type="model",
  exist_ok=True,
  token=hf_token,
  private=True
)
api.upload_folder(
  folder_path=new_model_name,
  repo_id=f"{user_name}/{new_model_name}",
  token=hf_token,
  commit_message="Initial Commit"
)